In [1]:
def read_fasta(file_path):
    """
    Reads a FASTA file and returns a dictionary where keys are the headers (chromosomes or enzyme names)
    and values are the corresponding sequences.
    """
    with open(file_path, 'r') as file:
        sequences = {}
        header = None # Initiating header to None
        sequence = []

        for line in file:
            line = line.strip()
            if line.startswith(">"):
                if header:
                    sequences[header] = ''.join(sequence)
                header = line[1:]  # Remove '>' character
                sequence = []
            else:
                sequence.append(line)
        
        # Add the last sequence
        if header:
            sequences[header] = ''.join(sequence)

    return sequences

In [36]:
def restriction_sites(file_path):
    """
    Reads the enzyme file and returns a dictionary where keys are the enzymes and values are a list
    consisting cutsite and the nick.
    """
    enzymes = read_fasta(file_path)
    for key in enzymes:
        nick = enzymes[key].find("|")
        enzymes[key] = [enzymes[key].replace("|", "").replace("N", "[ACTG]").replace("Y", "[CT]").replace("R", "[AG]"), nick]
        # Replace all the placeholders with the corresponding regex
    return enzymes

{'EcoRI': ['GAATTC', 3], 'HindIII': ['AAGCTT', 1], 'BsrI': ['ACTGG[ACTG]', 6]}

In [56]:
import re

def find_restriction_sites(genome, enzyme):
    """
    Finds restriction sites in the genome. Returns a dictionary where keys are headers (chromosomes)
    and values are lists of positions of the restriction site.
    """
    positions = {}
    restriction_site = restriction_sites("test_enzyme.fa")[enzyme]

    for chr, seq in genome.items():
        positions[chr] = [0]+ [restriction_site[1] + m.start() for m in re.finditer(restriction_site[0], seq)]
        # Find all the restriction sites and add the nick to the position, add the first position of the chromosome
        if positions[chr][-1] != len(seq):
            positions[chr].append(len(seq))
        # Add the last position of the chromosome

    return positions


In [57]:
find_restriction_sites(read_fasta("test_genome.fa"), "EcoRI")

{'I': [0, 3, 9, 15, 21, 27, 30], 'II': [0, 30], 'III': [0, 21, 27, 30]}

In [52]:
def calculate_distances(sites):
    """
    Calculates the distances between the restriction sites. Returns a dictionary where keys are
    headers (chromosomes) and values are lists of distances between the restriction sites.
    """
    distances = {}
    for chr, site in sites.items():
        distances[chr] = [site[i+1] - site[i] for i in range(len(site)-1)]
    return distances

In [53]:
calculate_distances(find_restriction_sites(read_fasta("test_genome.fa"), "EcoRI"))

{'I': [3, 6, 6, 6, 6, 3], 'II': [30], 'III': [21, 6, 3]}